### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File Threegram Selected Remain Word"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "Threegram_Remain_Word"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6  # 0.6
sample_num = 10  # 6

# youtube sentence condition
sent_time_start = False  # True, False
sent_time_start_time = 700  # sn
random_sample = True  # True, False

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    
    return  df_word_count

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [9]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [10]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [11]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [12]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_time_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

if random_sample:
    df_youtube_select = df_youtube_select.sample(len(df_youtube_select))
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    pass

df_youtube_select

,start_time,end_time,sentence,video_id
0,2160.240,2161.760,keşke bu örneği vermeseydin,kXuruorDTu8
1,1804.298,1805.759,damatlık getirmiştir allah bilir,3VYWCGkqPN0
2,3.020,6.340,sınav kaygısı bizim için çok önemli önümüzdeki...,yArv87XfINo
3,6751.600,6752.840,ben büyüdüm,g1Vag7OgCXw
4,341.759,342.271,çok kredi,iaYUCGqDbO0
...,...,...,...,...
3063204,2526.796,2528.717,şimdi belki bir süre ayrı kalacağız evet,Ejb6HObUkeI
3063205,879.450,881.583,halide geldi silahını çekti ve,H4WZF0yvzBg
3063206,1216.601,1223.881,yok eğer maksadın gerçekten kızınla görüşmekse...,2Hbt2lzGe90
3063207,5.229,9.787,bu insanların vergi borcu silinirken vergi bor...,YFaF9gRrm6g


In [13]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,threegram
0,ama şimdi olmaz
1,asla olmaz demek
2,ben gerçekten üzgünüm
3,benimle geliyor musun
4,bu küçük kız
5,bunun için teşekkürler
6,bütün gece mi
7,evet sanırım onları
8,geri ver şunu
9,hadi al şunu


In [ ]:
# Option
remove_videoid_list = list(set(df_word_group_select["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [14]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [15]:
len(search_list)

24

In [16]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,hadi al şunu,1586.851,1588.932,hadi al şunu allah aşkına al gidelim,M8xUPQyG_yg
1,hadi al şunu,5715.002,5716.278,sende sende vallahi sende hadi al şunu,tYBwrXTUMDQ
2,hadi al şunu,2011.594,2013.130,tamam hadi al şunu güney,yGLNsAuSPeM
3,hadi al şunu,3304.760,3307.360,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw
4,hadi al şunu,873.475,876.195,hadi al şunu da benimkini öne kaydırıver kanka,J0obG2KYeD4
5,hadi al şunu,3304.760,3307.360,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE
6,hadi al şunu,7783.760,7786.920,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc
7,hadi al şunu,2252.071,2254.696,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY
8,hadi al şunu,3324.000,3325.000,hadi al şunu,oE9qrxR7DpI
9,hadi al şunu,2252.071,2254.696,hadi al şunu al içeri götürelim bir müddet bur...,WfXIH3HS5b4


In [17]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,hadi al şunu,1586.851000,1587.602472,hadi al şunu allah aşkına al gidelim,M8xUPQyG_yg
1,hadi al şunu,5715.841474,5716.278000,sende sende vallahi sende hadi al şunu,tYBwrXTUMDQ
2,hadi al şunu,2011.914000,2012.810000,tamam hadi al şunu güney,yGLNsAuSPeM
3,hadi al şunu,3304.896842,3305.854737,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw
4,hadi al şunu,873.475000,874.243696,hadi al şunu da benimkini öne kaydırıver kanka,J0obG2KYeD4
5,hadi al şunu,3304.896842,3305.854737,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE
6,hadi al şunu,7784.747500,7786.130000,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc
7,hadi al şunu,2252.071000,2252.621403,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY
8,hadi al şunu,3324.000000,3325.000000,hadi al şunu,oE9qrxR7DpI
9,hadi al şunu,2252.071000,2252.621403,hadi al şunu al içeri götürelim bir müddet bur...,WfXIH3HS5b4


In [18]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,hadi al şunu,1586.251000,1588.202472,hadi al şunu allah aşkına al gidelim,M8xUPQyG_yg
1,hadi al şunu,5715.241474,5716.878000,sende sende vallahi sende hadi al şunu,tYBwrXTUMDQ
2,hadi al şunu,2011.314000,2013.410000,tamam hadi al şunu güney,yGLNsAuSPeM
3,hadi al şunu,3304.296842,3306.454737,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw
4,hadi al şunu,872.875000,874.843696,hadi al şunu da benimkini öne kaydırıver kanka,J0obG2KYeD4
5,hadi al şunu,3304.296842,3306.454737,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE
6,hadi al şunu,7784.147500,7786.730000,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc
7,hadi al şunu,2251.471000,2253.221403,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY
8,hadi al şunu,3323.400000,3325.600000,hadi al şunu,oE9qrxR7DpI
9,hadi al şunu,2251.471000,2253.221403,hadi al şunu al içeri götürelim bir müddet bur...,WfXIH3HS5b4


In [19]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,hadi al şunu,1586,1588,hadi al şunu allah aşkına al gidelim,M8xUPQyG_yg
1,hadi al şunu,5715,5717,sende sende vallahi sende hadi al şunu,tYBwrXTUMDQ
2,hadi al şunu,2011,2013,tamam hadi al şunu güney,yGLNsAuSPeM
3,hadi al şunu,3304,3306,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw
4,hadi al şunu,873,875,hadi al şunu da benimkini öne kaydırıver kanka,J0obG2KYeD4
5,hadi al şunu,3304,3306,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE
6,hadi al şunu,7784,7787,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc
7,hadi al şunu,2251,2253,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY
8,hadi al şunu,3323,3326,hadi al şunu,oE9qrxR7DpI
9,hadi al şunu,2251,2253,hadi al şunu al içeri götürelim bir müddet bur...,WfXIH3HS5b4


In [20]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,hadi al şunu,1586,1588,hadi al şunu allah aşkına al gidelim,M8xUPQyG_yg,https://www.youtube.com/watch?v=M8xUPQyG_yg&t=...
1,hadi al şunu,5715,5717,sende sende vallahi sende hadi al şunu,tYBwrXTUMDQ,https://www.youtube.com/watch?v=tYBwrXTUMDQ&t=...
2,hadi al şunu,2011,2013,tamam hadi al şunu güney,yGLNsAuSPeM,https://www.youtube.com/watch?v=yGLNsAuSPeM&t=...
3,hadi al şunu,3304,3306,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw,https://www.youtube.com/watch?v=Ri8OHAHmTJw&t=...
4,hadi al şunu,873,875,hadi al şunu da benimkini öne kaydırıver kanka,J0obG2KYeD4,https://www.youtube.com/watch?v=J0obG2KYeD4&t=...
5,hadi al şunu,3304,3306,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE,https://www.youtube.com/watch?v=lREyKzqpCjE&t=...
6,hadi al şunu,7784,7787,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc,https://www.youtube.com/watch?v=Q9bo4mKKszc&t=...
7,hadi al şunu,2251,2253,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY,https://www.youtube.com/watch?v=io3ajc3ysBY&t=...
8,hadi al şunu,3323,3326,hadi al şunu,oE9qrxR7DpI,https://www.youtube.com/watch?v=oE9qrxR7DpI&t=...
9,hadi al şunu,2251,2253,hadi al şunu al içeri götürelim bir müddet bur...,WfXIH3HS5b4,https://www.youtube.com/watch?v=WfXIH3HS5b4&t=...


In [21]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

8

In [22]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [23]:
search_all_set.difference(search_result_set)

{'ben gerçekten üzgünüm',
 'bunun için teşekkürler',
 'bütün gece mi',
 'evet sanırım onları',
 'geri ver şunu',
 'hadi ama dostum',
 'haydi git buradan',
 'herkes kendi geliyor',
 'hey neler oluyor',
 'oh hadi ama',
 'onunla birlikte değilim',
 'pekala hadi bakalım',
 'selam baba merhaba',
 'tabii eğer o',
 'tüm gece orada',
 'şimdi tekrar söyle'}

In [24]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,yoksa,2
1,olmaz,2
20,asla,1
19,benimle,1
18,bizi,1
17,bu,1
16,de,1
15,demek,1
14,geliyor,1
13,hadi,1


In [25]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [26]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [27]:
set_word.difference(set_count)

{'adam',
 'anne',
 'artık',
 'aslında',
 'aynı',
 'baba',
 'bak',
 'bakalım',
 'bana',
 'başka',
 'belki',
 'ben',
 'bence',
 'beni',
 'benim',
 'bile',
 'biliyor',
 'biliyorsun',
 'biliyorum',
 'bilmiyorum',
 'bir',
 'biraz',
 'biri',
 'birkaç',
 'birlikte',
 'biz',
 'bize',
 'bizim',
 'bugün',
 'buna',
 'bunu',
 'bunun',
 'burada',
 'buraya',
 'böyle',
 'bütün',
 'büyük',
 'da',
 'daha',
 'dakika',
 'devam',
 'değil',
 'değilim',
 'diye',
 'dostum',
 'doğru',
 'dur',
 'ederim',
 'efendim',
 'en',
 'et',
 'evet',
 'eğer',
 'fazla',
 'gece',
 'gel',
 'geldi',
 'gerek',
 'geri',
 'gerçek',
 'gerçekten',
 'gibi',
 'git',
 'göre',
 'gün',
 'güzel',
 'hakkında',
 'hala',
 'harika',
 'haydi',
 'hayır',
 'hemen',
 'her',
 'herkes',
 'hey',
 'hiç',
 'hiçbir',
 'iki',
 'ile',
 'ilk',
 'ister',
 'istiyorsun',
 'istiyorum',
 'iyi',
 'izin',
 'için',
 'içinde',
 'iş',
 'işte',
 'kadar',
 'kendi',
 'ki',
 'kimse',
 'kötü',
 'lazım',
 'lütfen',
 'merhaba',
 'misin',
 'mu',
 'mü',
 'mısın',
 'nasıl'

In [28]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [83]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [127]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_54_Threegram_Remain_Word_10_Youtube_0.6s_Timeshift_Result.xlsx']

In [129]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [130]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [131]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [132]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

,not_in_youtube_result
0,hadi al şunu
1,şimdi olmaz tamam
2,birlikte olacak mısın
3,haydi gidelim buradan
4,orada olmak istiyorum
...,...
168,üç adam vardı
169,bu küçük adam
170,bu sorun değil
171,gün bugün mü


In [133]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

173

In [134]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [135]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [136]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

,search_string,start_time,end_time,sentence,video_id
0,hadi al şunu,3304.760,3307.360,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE
1,hadi al şunu,2252.071,2254.696,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY
2,hadi al şunu,3324.000,3325.000,hadi al şunu,oE9qrxR7DpI
3,hadi al şunu,3304.760,3307.360,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw
4,hadi al şunu,7783.760,7786.920,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc
...,...,...,...,...,...
177,ne olduğunu biliyorum,1722.080,1731.530,oturalım bu da av köpeği yavrusu bunun cinsi n...,Tyx5100s0lY
178,ne olduğunu biliyorum,8138.315,8140.215,o sırrın ne olduğunu biliyorum,8Kq9E5l0SPM
179,üç adam vardı,1818.060,1819.340,üç adam vardı yanında,e3rIly3dY5E
180,gün bugün mü,1817.804,1818.991,o gün bugün mü,KnZuRjwzbus


In [137]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num

67

In [138]:
df_word_group_time_loc_result2 = word_group_time_loc(df_search_result2, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result2

,search_string,start_time,end_time,sentence,video_id
0,hadi al şunu,3304.896842,3305.854737,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE
1,hadi al şunu,2252.071000,2252.621403,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY
2,hadi al şunu,3324.000000,3325.000000,hadi al şunu,oE9qrxR7DpI
3,hadi al şunu,3304.896842,3305.854737,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw
4,hadi al şunu,7784.747500,7786.130000,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc
...,...,...,...,...,...
177,ne olduğunu biliyorum,1727.480000,1730.302727,oturalım bu da av köpeği yavrusu bunun cinsi n...,Tyx5100s0lY
178,ne olduğunu biliyorum,8138.821667,8140.215000,o sırrın ne olduğunu biliyorum,8Kq9E5l0SPM
179,üç adam vardı,1818.060000,1818.913333,üç adam vardı yanında,e3rIly3dY5E
180,gün bugün mü,1817.888786,1818.991000,o gün bugün mü,KnZuRjwzbus


In [139]:
df_word_group_time_loc_result2.start_time = df_word_group_time_loc_result2.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result2.end_time = df_word_group_time_loc_result2.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result2

,search_string,start_time,end_time,sentence,video_id
0,hadi al şunu,3304.296842,3306.454737,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE
1,hadi al şunu,2251.471000,2253.221403,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY
2,hadi al şunu,3323.400000,3325.600000,hadi al şunu,oE9qrxR7DpI
3,hadi al şunu,3304.296842,3306.454737,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw
4,hadi al şunu,7784.147500,7786.730000,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc
...,...,...,...,...,...
177,ne olduğunu biliyorum,1726.880000,1730.902727,oturalım bu da av köpeği yavrusu bunun cinsi n...,Tyx5100s0lY
178,ne olduğunu biliyorum,8138.221667,8140.815000,o sırrın ne olduğunu biliyorum,8Kq9E5l0SPM
179,üç adam vardı,1817.460000,1819.513333,üç adam vardı yanında,e3rIly3dY5E
180,gün bugün mü,1817.288786,1819.591000,o gün bugün mü,KnZuRjwzbus


In [140]:
df_word_group_time_loc_result2["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result2['video_id'].map(str)+"&t="+df_word_group_time_loc_result2['start_time'].map(str)+"s"
df_word_group_time_loc_result2

,search_string,start_time,end_time,sentence,video_id,video_url
0,hadi al şunu,3304.296842,3306.454737,ee hadi al şunu hayvanlara yaş mama al,lREyKzqpCjE,https://www.youtube.com/watch?v=lREyKzqpCjE&t=...
1,hadi al şunu,2251.471000,2253.221403,hadi al şunu al içeri götürelim bir müddet bur...,io3ajc3ysBY,https://www.youtube.com/watch?v=io3ajc3ysBY&t=...
2,hadi al şunu,3323.400000,3325.600000,hadi al şunu,oE9qrxR7DpI,https://www.youtube.com/watch?v=oE9qrxR7DpI&t=...
3,hadi al şunu,3304.296842,3306.454737,ee hadi al şunu hayvanlara yaş mama al,Ri8OHAHmTJw,https://www.youtube.com/watch?v=Ri8OHAHmTJw&t=...
4,hadi al şunu,7784.147500,7786.730000,neyse hadi hadi al şunu fişe tak,Q9bo4mKKszc,https://www.youtube.com/watch?v=Q9bo4mKKszc&t=...
...,...,...,...,...,...,...
177,ne olduğunu biliyorum,1726.880000,1730.902727,oturalım bu da av köpeği yavrusu bunun cinsi n...,Tyx5100s0lY,https://www.youtube.com/watch?v=Tyx5100s0lY&t=...
178,ne olduğunu biliyorum,8138.221667,8140.815000,o sırrın ne olduğunu biliyorum,8Kq9E5l0SPM,https://www.youtube.com/watch?v=8Kq9E5l0SPM&t=...
179,üç adam vardı,1817.460000,1819.513333,üç adam vardı yanında,e3rIly3dY5E,https://www.youtube.com/watch?v=e3rIly3dY5E&t=...
180,gün bugün mü,1817.288786,1819.591000,o gün bugün mü,KnZuRjwzbus,https://www.youtube.com/watch?v=KnZuRjwzbus&t=...


In [141]:
search_result_num2 = df_word_group_time_loc_result2["search_string"].nunique()
search_result_num2

67

In [142]:
df_word_group_time_loc_result2.to_excel(f"{lang_folder.capitalize()}_{search_result_num2}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result2.xlsx", index=False) 